In [4]:
import random
import nltk 
from nltk import bigrams, FreqDist, ConditionalFreqDist

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
import os
import string

## Keep your training documents in a folder named 'data'
input_data_dir = "data"

# String of punctuation without the full stop
punctuation = string.punctuation.replace('.', '')  # Retain the full stop

def is_hidden(filepath):
    return os.path.basename(filepath).startswith('.')

text_data=""
for filename in os.listdir(input_data_dir):
    filepath = os.path.join(input_data_dir, filename)
    if not is_hidden(filepath):
        with open(filepath) as infile:
            for line in infile:
                if line.strip():  # Check if line is not just whitespace
                    # Remove all punctuation except full stops
                    for char in punctuation:
                        line = line.replace(char, '')
                    text_data += line

In [11]:
len(text_data)

8441833

In [13]:
# Tokenize the text into words
# Lowercasing for consistency
words = nltk.word_tokenize(text_data.lower())

# Generate bigrams
bi_grams = list(bigrams(words))

# Calculate frequency distribution for each bigram
bi_gram_freq_dist = FreqDist(bi_grams)

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/codespace/nltk_data'
    - '/home/codespace/.python/current/nltk_data'
    - '/home/codespace/.python/current/share/nltk_data'
    - '/home/codespace/.python/current/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
from itertools import islice
# Print the first five elements of the dictionary
first_five_items = list(islice(bi_gram_freq_dist.items(), 5))
for item in first_five_items:
    print(item)

In [ ]:
# Compute conditional frequency distribution of bigrams
bi_gram_freq = ConditionalFreqDist(bi_grams)

In [ ]:
bi_gram_freq['natural']

In [ ]:
import heapq

topk=3
# Create a dictionary to hold the top topk bigrams for each first word
top_bigrams_per_first_word = {}

# Iterate over the bigram frequency distribution
for (first_word, second_word), freq in bi_gram_freq_dist.items():
    # Initialize an empty heap for the first_word if it doesn't exist
    if first_word not in top_bigrams_per_first_word:
        top_bigrams_per_first_word[first_word] = []

    # Add to the heap and maintain top topk
    heapq.heappush(top_bigrams_per_first_word[first_word],
                   (freq, second_word))
    if len(top_bigrams_per_first_word[first_word]) > topk:
        heapq.heappop(top_bigrams_per_first_word[first_word])


In [ ]:
top_bigrams_per_first_word['natural']

In [ ]:
# Convert the heap to a simple list for each first word
for first_word in top_bigrams_per_first_word:
    sorted_bigrams = sorted(
        top_bigrams_per_first_word[first_word], reverse=True)
    top_bigrams_list = []
    for freq, second_word in sorted_bigrams:
        top_bigrams_list.append(second_word)
    top_bigrams_per_first_word[first_word] = top_bigrams_list

# Use these filtered bigrams to create a ConditionalFreqDist
filtered_bi_grams = []
for first_word in top_bigrams_per_first_word:
    for second_word in top_bigrams_per_first_word[first_word]:
        filtered_bi_grams.append((first_word, second_word))

bi_gram_freq = ConditionalFreqDist(filtered_bi_grams)

In [ ]:
def generate_sentence(word, num_words):
    word =word.lower()
    for _ in range(num_words):
        print(word, end=' ')
        next_words = [item for item, freq in bi_gram_freq[word].items()]
        if len(next_words) > 0:
            # Randomly choose a next word
            word = random.choice(next_words)
        else:
            break  # Break if the word has no following words
    print()

In [ ]:
generate_sentence('Asia', 100)

# Using a simple bigram model to create a slm

In [2]:
text_string = """Here’s an expanded version of your paragraph, extended to approximately 300–400 words:

Hello, World! Everything here is beautiful, from the vibrant colors of the flowers to the serene sounds of nature. The sun casts a warm glow over the landscape, illuminating every corner with its golden rays, creating a mesmerizing display of light and shadow. As I walk through this picturesque setting, I can’t help but notice the delicate dance of butterflies fluttering from bloom to bloom, their wings painted in hues of blue, yellow, and orange. Each flower stands proudly, boasting its unique beauty, while the lush greenery provides a stunning backdrop that enhances the vivid colors of nature.

Birds sing sweet melodies, adding to the symphony of life all around. Their songs echo through the air, harmonizing with the gentle rustling of leaves in the breeze. The tranquil sounds of nature serve as a soothing soundtrack, inviting moments of reflection and appreciation for the world we inhabit. Nearby, a small stream flows gracefully, its clear waters sparkling in the sunlight. The soothing sound of water trickling over smooth stones creates a sense of calm, making it a perfect spot for a peaceful retreat.

Children laugh and play, their joy infectious as they run freely in the open fields. Their laughter rings like music, uplifting spirits and spreading happiness. They chase after each other, their carefree energy radiating a sense of pure delight. In this moment, time seems to stand still, allowing everyone to appreciate the simple pleasures of existence. The warm sun on my skin, the gentle breeze in my hair, and the comforting scents of earth and flowers fill my senses, grounding me in the present.

Together, we create memories that last a lifetime, reminding us of the beauty and wonder that surrounds us every day. It’s in these small moments—savoring the beauty of nature, relishing laughter, and enjoying time with loved ones—that we find true happiness. As the day unfolds, I realize that life is a series of such moments, each one precious and unique. Embracing them fully helps us cultivate gratitude and a deeper connection to the world, encouraging us to cherish the simple joys and marvel at the wonders that life has to offer.

Feel free to adjust any parts of the paragraph as needed!"""

In [4]:
import random

def tokenize(text):
    return text.replace(',', '').replace('!', '').split()

def generate_bigrams(words):
    return [(words[i].lower(), words[i + 1].lower()) for i in range(len(words) - 1)]

def create_bigram_model(bigrams):
    bigram_model = {}
    for w1, w2 in bigrams:
        if w1 not in bigram_model:
            bigram_model[w1] = {}
        bigram_model[w1][w2] = bigram_model[w1].get(w2, 0) + 1
    for w1, next_words in bigram_model.items():
        total_count = sum(next_words.values())
        bigram_model[w1] = {w2: count / total_count for w2, count in next_words.items()}
    return bigram_model

def generate_words(bigram_model, start_word, count, threshold):
    current_word = start_word.lower()
    generated_words = []
    for _ in range(count):
        next_words = bigram_model.get(current_word, {})
        valid_next_words = {word: prob for word, prob in next_words.items() if prob > threshold}
        if not valid_next_words:
            break
        chosen_word = random.choices(list(valid_next_words.keys()), weights=list(valid_next_words.values()))[0]
        generated_words.append(chosen_word)
        current_word = chosen_word
    return generated_words

tokenized_words = tokenize(text_string)
bigrams = generate_bigrams(tokenized_words)
bigram_model = create_bigram_model(bigrams)

start_word = "hello"
threshold = 0.2

new_words = generate_words(bigram_model, start_word, 10, threshold)
print("Generated words:", new_words)


Generated words: ['world', 'everything', 'here', 'is', 'a']
